## CMPE 346
## Natural Language Processing




---


### Term Project Report

### Spell Checker For Turkish Language Using Norvig’s Spell Checker Algorithm




---


#### By
#### Group Members
#### Ali BEYAZ - 113200050
#### Burak GÜLER - 11575012
#### Efe HEPDOĞAN - 112200062

---

Dataset Linki: https://drive.google.com/drive/folders/1ng8TNk6EY0fwJDJpzS1J2kLjra-KWrKc?usp=sharing



In [ ]:
import re
from collections import Counter

In [ ]:
def words(text): return re.findall(r'\w+', text.lower())
#WORDS = Counter(words(open('/content/drive/My Drive/Courses 2020/cmpe 346/term-project/data_X.txt').read()))
WORDS = Counter(words(open('/content/drive/My Drive/Courses 2020/cmpe 346/term-project/final-dataset.txt').read()))

In [ ]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcçdefghıijklmnoöpqrsştuüvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
len(WORDS)

1277621

In [ ]:
sum(WORDS.values())

8312849

In [ ]:
WORDS.most_common(10)

[('ve', 136599),
 ('bir', 117191),
 ('da', 76104),
 ('de', 68571),
 ('bu', 63819),
 ('nin', 49445),
 ('için', 43832),
 ('türkiye', 39201),
 ('ın', 33799),
 ('ile', 32622)]

In [ ]:
known(edits1('bardklar'))

{'baraklar', 'bardaklar', 'bardalar'}

In [ ]:
known(edits1('düşüdnüm'))

{'düşündüm', 'düşünüm'}

In [ ]:
known(edits2('bardklar'))

{'araklar',
 'ardalar',
 'arklar',
 'arıklar',
 'bacaklar',
 'balıklar',
 'bandolar',
 'banklar',
 'barajlar',
 'barakalar',
 'barakla',
 'baraklar',
 'baranlar',
 'barcılar',
 'bardakla',
 'bardaklar',
 'bardaklara',
 'bardakları',
 'bardalar',
 'barkla',
 'barkları',
 'barkodlar',
 'barlar',
 'barokla',
 'barolar',
 'baronlar',
 'barsaklar',
 'barsalar',
 'bartular',
 'barutlar',
 'barçalar',
 'barışlar',
 'basaklar',
 'basklar',
 'bataklar',
 'batıklar',
 'baydılar',
 'bayraklar',
 'başaklar',
 'boraklar',
 'bordalar',
 'bordolar',
 'buraklar',
 'buruklar',
 'farklar',
 'gardlar',
 'kardılar',
 'karıklar',
 'marklar',
 'nardılar',
 'parklar',
 'sardılar',
 'sarıklar',
 'taraklar',
 'tarıklar',
 'varaklar',
 'vardıklar',
 'vardılar',
 'yaraklar',
 'yardılar',
 'yarıklar',
 'zardılar',
 'çardaklar',
 'çarklar',
 'çarıklar'}

In [ ]:
WORDS.most_common(10)

[('ve', 136599),
 ('bir', 117191),
 ('da', 76104),
 ('de', 68571),
 ('bu', 63819),
 ('nin', 49445),
 ('için', 43832),
 ('türkiye', 39201),
 ('ın', 33799),
 ('ile', 32622)]

In [ ]:
P('ve')

0.016432272497671978

In [ ]:
P('bilinirlik')

0.0

In [ ]:
def unit_tests():
    assert correction('çözüsüzlüğün') == 'çözümsüzlüğün'             # insert
    assert correction('balşayan') == 'başlayan'                      # replace 2
    assert correction('çözebılirdik') == 'çözebilirdik'              # replace
    assert correction('belirlenmemşti') == 'belirlenmemişti'         # insert 
    assert correction('imdiadına') == 'imdadına'                     # delete
    assert correction('şühpelenen') =='şüphelenen'                   # transpose
    assert correction('daeiresi') =='dairesi'                        # transpose + delete
    assert correction('itibaren') == 'itibaren'                      # known
    assert len(WORDS) == 1277621
    assert sum(WORDS.values()) == 8312849
    assert WORDS.most_common(10) == [
           ('ve', 136599),
           ('bir', 117191),
           ('da', 76104),
           ('de', 68571),
           ('bu', 63819),
           ('nin', 49445),
           ('için', 43832),
           ('türkiye', 39201),
           ('ın', 33799),
           ('ile', 32622)]
    assert WORDS['ve'] == 136599
    assert P('bilinirlik') == 0
    assert 0.01 < P('ve') < 0.02
    return 'unit_tests pass'

def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())
spelltest(Testset(open('/content/drive/My Drive/Courses 2020/cmpe 346/term-project/test-case3.1.txt'))) # Data testset

unit_tests pass
61% of 895 correct (1% unknown) at 33 words per second 
